# Modelagem preditiva
_Machine Learning_

---

## Sumário

1. **Importação de bibliotecas**
2. **Carregamento das bases**

<br>

---

<br>

## 1. Importação de bibliotecas

In [ ]:
# Importação de pacotes e definição de parâmetros globais

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import seaborn as sns
import warnings
import gc

In [ ]:
# Configurações para exibição de dados no Jupyter Notebook

# Configurar opção para exibir todas as linhas do Dataframe
pd.set_option('display.max_rows', None)

# Configurar para exibir o conteúdo completo das colunas
pd.set_option('display.max_colwidth', None)

# Configurar a supressão de mensagens de aviso durante a execução
warnings.filterwarnings('ignore')

# Configurar estilo dos gráficos do seaborn
sns.set_style('whitegrid')

## 2. Carregamento das bases

In [ ]:
# Efetuando a limpeza da memória antes do carregamento dos dados
print(f'\nQuantidade de objetos removidos da memória: {gc.collect()}')

In [ ]:
# Criando um dataframe a partir do arquivo train_001.csv
df_train = pd.read_csv('dados/train_001.csv', sep=',')
print('\nDATAFRAME: df_train')
df_train.head()

In [ ]:
# Criando um dataframe a partir do arquivo validation_001.csv
df_validation = pd.read_csv('dados/validation_001.csv', sep=',')
print('\nDATAFRAME: df_validation')
df_validation.head()